## working with spacy

https://spacy.io/api/textcategorizer

In [ ]:
# can convert to .spacy like this:
!spacy convert ../../data/dreaddit/train -c conllu -n 5 ../../data/dreaddit/output

ℹ Grouping every 5 sentences into a document.
✔ Generated output file (1 documents): ..\..\data\dreaddit\output\1421.spacy
ℹ Grouping every 5 sentences into a document.
✔ Generated output file (2 documents): ..\..\data\dreaddit\output\1517.spacy
ℹ Grouping every 5 sentences into a document.
✔ Generated output file (1 documents): ..\..\data\dreaddit\output\165.spacy
ℹ Grouping every 5 sentences into a document.
✔ Generated output file (1 documents): ..\..\data\dreaddit\output\17554.spacy
ℹ Grouping every 5 sentences into a document.
✔ Generated output file (1 documents): ..\..\data\dreaddit\output\239.spacy
ℹ Grouping every 5 sentences into a document.
✔ Generated output file (1 documents): ..\..\data\dreaddit\output\2606.spacy
ℹ Grouping every 5 sentences into a document.
✔ Generated output file (1 documents): ..\..\data\dreaddit\output\33053.spacy
ℹ Grouping every 5 sentences into a document.
✔ Generated output file (1 documents): ..\..\data\dreaddit\output\33181.spacy
ℹ Grouping ever

In [ ]:
display(df.head())
display(df.columns)

subreddit post_id sentence_range  \
0              ptsd  8601tu       (15, 20)   
1        assistance  8lbrx9         (0, 5)   
2              ptsd  9ch1zh       (15, 20)   
3     relationships  7rorpp        [5, 10]   
4  survivorsofabuse  9p2gbc         [0, 5]   

                                                text     id  label  \
0  He said he had not felt that way before, sugge...  33181      1   
1  Hey there r/assistance, Not sure if this is th...   2606      0   
2  My mom then hit me with the newspaper and it s...  38816      1   
3  until i met my new boyfriend, he is amazing, h...    239      1   
4  October is Domestic Violence Awareness Month a...   1421      1   

   confidence  social_timestamp  social_karma  syntax_ari  ...  \
0         0.8        1521614353             5    1.806818  ...   
1         1.0        1527009817             4    9.429737  ...   
2         0.8        1535935605             2    7.769821  ...   
3         0.6        1516429555             0    2.667798  ...   
4         0.8        1539809005            24    7.554238  ...   

   lex_dal_min_pleasantness  lex_dal_min_activation  lex_dal_min_imagery  \
0                     1.000                  1.1250                  1.0   
1                     1.125                  1.0000                  1.0   
2                     1.000                  1.1429                  1.0   
3                     1.000                  1.1250                  1.0   
4                     1.000                  1.1250                  1.0   

   lex_dal_avg_activation  lex_dal_avg_imagery  lex_dal_avg_pleasantness  \
0                 1.77000              1.52211                   1.89556   
1                 1.69586              1.62045                   1.88919   
2                 1.83088              1.58108                   1.85828   
3                 1.75356              1.52114                   1.98848   
4                 1.77644              1.64872                   1.81456   

   social_upvote_ratio  social_num_comments  syntax_fk_grade  sentiment  
0                 0.86                    1         3.253573  -0.002742  
1                 0.65                    2         8.828316   0.292857  
2                 0.67                    0         7.841667   0.011894  
3                 0.50                    5         4.104027   0.141671  
4                 1.00                    1         7.910952  -0.204167  

[5 rows x 116 columns]

Index(['subreddit', 'post_id', 'sentence_range', 'text', 'id', 'label',
       'confidence', 'social_timestamp', 'social_karma', 'syntax_ari',
       ...
       'lex_dal_min_pleasantness', 'lex_dal_min_activation',
       'lex_dal_min_imagery', 'lex_dal_avg_activation', 'lex_dal_avg_imagery',
       'lex_dal_avg_pleasantness', 'social_upvote_ratio',
       'social_num_comments', 'syntax_fk_grade', 'sentiment'],
      dtype='object', length=116)

In [ ]:
# small subset in format
data = []

for idx in tqdm(range(0, len(df))):
    row = df.iloc[idx]
    data += [(row.text, row.label)]

100%|██████████| 10/10 [00:00<00:00, 4981.36it/s]


In [ ]:
# ...
import spacy
from spacy.tokens import DocBin

from spacy_conll import init_parser

nlp = init_parser("en",
                  "stanza",
                  parser_opts={"use_gpu": True, "verbose": True},
                  include_headers=True)

# the DocBin will store the example documents
db = DocBin()

for sentence, label in nlp.pipe(data,as_tuples=True):
    print(label)
    doc = nlp(sentence)
    doc.cats["STRESS"] = label == 1
    doc.cats["CHILL"] = label == 0
    db.add(doc)


db.to_disk("./train.spacy")

2023-10-30 00:46:28 INFO: Downloading default packages for language: en (English) ...
2023-10-30 00:46:29 INFO: File exists: C:\Users\luisk\stanza_resources\en\default.zip
2023-10-30 00:46:33 INFO: Finished downloading models and saved to C:\Users\luisk\stanza_resources.
2023-10-30 00:46:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-10-30 00:46:34 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-10-30 00:46:34 WARNING: GPU requested, but is not available!
2023-10-30 00:46:34 I

1
0
1
1
1
1
0
1
1
1


In [ ]:
# Deserialize later, e.g. in a new process
doc_bin = DocBin().from_disk("./train.spacy")
for d in doc_bin.get_docs(nlp.vocab):
    conll = doc._.conll_str
    print(f'{d.cats}\t{d.text}')
c = d._.conll_str
print(c)

{'STRESS': True, 'CHILL': False}	He said he had not felt that way before, suggeted I go rest and so ..TRIGGER AHEAD IF YOUI'RE A HYPOCONDRIAC LIKE ME: i decide to look up "feelings of doom" in hopes of maybe getting sucked into some rabbit hole of ludicrous conspiracy, a stupid "are you psychic" test or new age b.s., something I could even laugh at down the road. No, I ended up reading that this sense of doom can be indicative of various health ailments; one of which I am prone to.. So on top of my "doom" to my gloom..I am now f'n worried about my heart. I do happen to have a physical in 48 hours.
{'STRESS': False, 'CHILL': True}	Hey there r/assistance, Not sure if this is the right place to post this.. but here goes =) I'm currently a student intern at Sandia National Labs and working on a survey to help improve our marketing outreach efforts at the many schools we recruit at around the country. We're looking for current undergrad/grad STEM students so if you're a STEM student or know

In [ ]:
import spacy
from spacy_conll import ConllFormatter

nlp = spacy.load('en_core_web_sm')
conllformatter = ConllFormatter(nlp, ext_names={'conll_pd': 'pandas'},
                                conversion_maps={'lemma': {'-PRON-': 'PRON'}})
nlp.add_pipe(conllformatter, after='parser')

doc_bin = DocBin().from_disk("./train.spacy")

doc = nlp('The quick brown fox jumps over the lazy dog.')
print(doc._.pandas)

In [ ]:
from spacy_conll import init_parser

nlp = init_parser("en",
                  "stanza",
                  parser_opts={"use_gpu": True, "verbose": True},
                  include_headers=True)

doc = nlp("A cookie is a baked or cooked food that is typically small, flat and sweet. It usually contains flour, sugar and some type of oil or fat.")
conll = doc._.conll_str
print(conll)

2023-10-30 00:45:14 INFO: Downloading default packages for language: en (English) ...
2023-10-30 00:45:15 INFO: File exists: C:\Users\luisk\stanza_resources\en\default.zip
2023-10-30 00:45:18 INFO: Finished downloading models and saved to C:\Users\luisk\stanza_resources.
2023-10-30 00:45:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-10-30 00:45:20 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-10-30 00:45:20 WARNING: GPU requested, but is not available!
2023-10-30 00:45:20 I

# sent_id = 1
# text = A cookie is a baked or cooked food that is typically small, flat and sweet.
1	A	a	DET	DT	Definite=Ind|PronType=Art	2	det	_	_
2	cookie	cookie	NOUN	NN	Number=Sing	8	nsubj	_	_
3	is	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	8	cop	_	_
4	a	a	DET	DT	Definite=Ind|PronType=Art	8	det	_	_
5	baked	bake	VERB	VBN	Tense=Past|VerbForm=Part	8	amod	_	_
6	or	or	CCONJ	CC	_	7	cc	_	_
7	cooked	cook	VERB	VBN	Tense=Past|VerbForm=Part	5	conj	_	_
8	food	food	NOUN	NN	Number=Sing	0	root	_	_
9	that	that	PRON	WDT	PronType=Rel	12	nsubj	_	_
10	is	be	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	12	cop	_	_
11	typically	typically	ADV	RB	_	12	advmod	_	_
12	small	small	ADJ	JJ	Degree=Pos	8	acl:relcl	_	SpaceAfter=No
13	,	,	PUNCT	,	_	14	punct	_	_
14	flat	flat	ADJ	JJ	Degree=Pos	12	conj	_	_
15	and	and	CCONJ	CC	_	16	cc	_	_
16	sweet	sweet	ADJ	JJ	Degree=Pos	12	conj	_	SpaceAfter=No
17	.	.	PUNCT	.	_	8	punct	_	_

# sent_id = 2
# text = It usually contains flour, sugar and